
$$
Sets
$$

$$
t = \{1, 2, 3, 4, \ldots, 12\}
$$


$$
Parameters
$$

$$
D_t: \text{demand in month t (in tons)}
$$

$$
c_t: \text{inventory holding cost in month t}
$$

$$
c_t = \$100 \text{ per ton}
$$

$$
r_t: \text{regular time labor cost per 100 employees per hour in month t}
$$

$$
r_t = \$10 \times 100 = \$1000/\text{month}
$$

$$
o_t: \text{overtime labor cost per 100 employees per hour in month } t
$$

$$
o_t = \$15 × 100= \$1500/\text{month}
$$

$$
p_t: \text{cost of production in month } t
$$

$$
p_t = \$1000 \text{ per ton}
$$

$$
Decision Variables
$$

$$
R_t : \text{regular hours worked in month } t
$$


$$
W_t : \text{workforce level in month } t
$$

$$
I_t : \text{inventory level at the end of month } t
$$

$$
P_t : \text{production level in month } t
$$

$$
O_t : \text{number of hours worked in overtime in month t}
$$


$$
 Model
$$


$$
\min \sum c_t \cdot I_t + r_t \cdot R_t + o_t \cdot O_t + p_t \cdot P_t
$$

$$
s.t.
$$


$$
I_t = I_{t-1} + P_t - D_t
$$

$$
P_t = R_t + O_t
$$

$$
I_0 = 150
$$

$$
I_{12} = 150
$$

$$
I_t \geq 100 \quad \text{for} \quad t \in \{1, 2, 3, \dots, 11\}
$$

$$
R_t \leq 320
$$

$$
O_t \leq 40
$$


$$
R_t, I_t, P_t, O_t \geq 0
$$

In [ ]:
!pip install pulp

In [ ]:
from pulp import *

# Question 1

In [ ]:
# Months
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# Months for inventory since I0 exists
months_inventory = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# Dt for each month
demand = {

1 : 280,
2 : 301,
3 : 277,
4 : 310,
5 : 285,
6 : 278,
7 : 291,
8 : 220,
9 : 304,
10 : 295,
11 : 302,
12 : 297
}

# rt
regular_time_cost = 1000

# ot
overtime_time_cost = 1500

# ct
inventory_holding_cost = 100

# pt
production_cost = 1000

**Problem is a minimization problem:**

In [ ]:
model = LpProblem("Aggregate-Planning-Problem", LpMinimize)

**Decision variables are defined and greater than zero:**

In [ ]:
# It
inventory_vars = LpVariable.dicts("Inventory", months_inventory, 0)
# Pt
production_vars = LpVariable.dicts("Production", months, 0)
# Ot
overtime_hours = LpVariable.dicts("Overtime", months, 0)
# Rt
regular_hours = LpVariable.dicts("Hours", months, 0)
# Wt
workforce_vars = 100

**Adding the objective function:**

In [ ]:
Tlabor    = lpSum(regular_time_cost * regular_hours[i] for i in months) # regular hour cost each month
Tinv      = lpSum(inventory_holding_cost * inventory_vars[i] for i in months_inventory) # inventory carry cost each month
Tovertime = lpSum(overtime_time_cost * overtime_hours[i] for i in months) # overtime cost each month
Tprod     = lpSum(production_cost * production_vars[i] for i in months) # production cost each month

model += Tlabor + Tinv + Tovertime + Tprod

**Adding the constraints:**

 **Inventory balance**

In [ ]:
for i in months_inventory:

    if i == 0:
        model += inventory_vars[i] == 150,  f"Inventory 0" # Starting inventory is 150
        model += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand[i + 1],  f"Inventory Balance {i}" # Inventory Balance for January
    elif i == 12:
        model += inventory_vars[i] == 150,  f"Inventory Balance {i}" # Ending inventory is 150
    else:
        model += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand[i+1], f"Inventory Balance {i}" # Inventory Balance for all months except January and December

**Production Workforce Relation**

 Each ton of detergent requires one hour of operation of the line. For each regular hour or overtime hour worked, one ton of detergent is being produced.

In [ ]:
for i in months:

    model += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"

**Inventory Level for Intermediate Months**

During intermediate months, Q&H wants at least 100 tons of inventory and
wants to consider only plans without any stockouts.   

In [ ]:
for i in months:

    if i != 12:

        model += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

**Work Hours**

The plant works 20  days a month, two shifts a day, and eight hours a shift of regular time, which translates to a maximum of 320 hours of regular hours per month.

In [ ]:
for i in months:

    model += regular_hours[i] <= 320, f"Work Hours Constraint {i}"

**Overtime Limit**

Overtime is restricted to 20 hours per employee each month. Accounted for 2 shifts, a maximum of 40 hours of overtime hours can be attained.

In [ ]:
for i in months:

    model += 40 >= overtime_hours[i], f"Overtime Constraint {i}"

**Solving the problem and checking for optimality:**

In [ ]:
# Solve the problem
model.solve()
LpStatus[model.status]

'Optimal'

In [ ]:
print("Optimal objective function value = ", value(model.objective))

Optimal objective function value =  7023600.0


**Variables and their values in the solution:**

In [ ]:
for v in model.variables():
    print(v.name, "=", v.varValue)

Hours_1 = 230.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 301.0
Hours_3 = 277.0
Hours_4 = 310.0
Hours_5 = 285.0
Hours_6 = 278.0
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 100.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 100.0
Inventory_3 = 100.0
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 0.0
Overtime_3 = 0.0
Overtime_4 = 0.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 230.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 301.0
Production_3 = 277.0
Production_4 = 310.0
Production_5 = 285.0
Production_6 = 278.0
Production_7 = 291.0
Production_8 = 220.0
Production_9 = 304.0


# Question 2

$$
 \text{New Parameters}
$$

$$
s_t: \text{sales price}
$$

$$
s_t = $2600 \text{ per ton}
$$

$$
\text{New Model}
$$

$$
\max \sum s_t ⋅ D_t - c_t \cdot I_t - r_t \cdot R_t - o_t \cdot O_t - p_t \cdot P_t
$$

$$
s.t. \text{same constraints}
$$


Total sales is equal to st * Dt

In [ ]:
Tsales = lpSum(2600 * demand[i] for i in months)

model2 = LpProblem("Aggregate-Planning-Problem", LpMaximize)
model2 += Tsales - Tlabor - Tinv - Tovertime - Tprod

In [ ]:
for i in months_inventory:

    if i == 0:
        model2 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model2 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand[i + 1]
    elif i == 12:
        model2 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model2 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand[i+1], f"Inventory Balance {i}"


for i in months:
    model2 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"


for i in months:
    if i != 12:
        model2 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

for i in months:
    model2 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model2 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
# Solve the problem
model2.solve()
LpStatus[model2.status]

'Optimal'

In [ ]:
print("Optimal objective function value = ", value(model2.objective))

Optimal objective function value =  1920400.0


In [ ]:
for v in model2.variables():
    print(v.name, "=", v.varValue)

Hours_1 = 230.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 301.0
Hours_3 = 277.0
Hours_4 = 310.0
Hours_5 = 285.0
Hours_6 = 278.0
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 100.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 100.0
Inventory_3 = 100.0
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 0.0
Overtime_3 = 0.0
Overtime_4 = 0.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 230.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 301.0
Production_3 = 277.0
Production_4 = 310.0
Production_5 = 285.0
Production_6 = 278.0
Production_7 = 291.0
Production_8 = 220.0
Production_9 = 304.0


# Question 3

$$
\text{Parameter Update}
$$

$$
s_4 = $2600 - $260 = $2340
$$

$$
D_4 = D_4 \times 150\% + D_5 \times 20\% + D_6 \times 20\%
$$

$$
D_5 = D_5 \times 80\%
$$

$$
D_7 = D_7 \times 80\%
$$

In [ ]:
demand_q3 = {

1 : 280,
2 : 301,
3 : 277,
4 : 310 * 1.5 + 285 * 0.2 + 278 * 0.2,
5 : 285 * 0.8,
6 : 278 * 0.8,
7 : 291,
8 : 220,
9 : 304,
10 : 295,
11 : 302,
12 : 297
}

st_q3 = {

1 : 2600,
2 : 2600,
3 : 2600,
4 : 2340,
5 : 2600,
6 : 2600,
7 : 2600,
8 : 2600,
9 : 2600,
10 : 2600,
11 : 2600,
12 : 2600
}

In [ ]:
Tsales_q3 = lpSum(st_q3[i] * demand_q3[i] for i in months)

model3 = LpProblem("Aggregate-Planning-Problem", LpMaximize)
model3 += Tsales_q3 - Tlabor - Tinv - Tovertime - Tprod

In [ ]:
for i in months_inventory:

    if i == 0:
        model3 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model3 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand[i + 1]
    elif i == 12:
        model3 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model3 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand_q3[i+1], f"Inventory Balance {i}"


for i in months:

    model3 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"


for i in months:

    if i != 12:

        model3 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"


for i in months:

    model3 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:

    model3 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model3.solve()
LpStatus[model3.status]

'Optimal'

In [ ]:
print("Optimal objective function value = ", value(model3.objective))

Optimal objective function value =  1766204.0


In [ ]:
for v in model3.variables():
    print(v.name, "=", v.varValue)

Hours_1 = 320.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 320.0
Hours_3 = 320.0
Hours_4 = 320.0
Hours_5 = 228.0
Hours_6 = 222.4
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 190.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 234.6
Inventory_3 = 317.6
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 25.6
Overtime_3 = 40.0
Overtime_4 = 40.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 320.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 345.6
Production_3 = 360.0
Production_4 = 360.0
Production_5 = 228.0
Production_6 = 222.4
Production_7 = 291.0
Production_8 = 220.0
Production_9 = 304.0


# Question 4

$$
\text{Parameter Update}
$$

$$
s_8 = $2600 - $260 = $2340
$$

$$
D_8 = D_8 \times 150\% + D_9 \times 20\% + D_{10} \times 20\%
$$

$$
D_9 = D_9 \times 80\%
$$

$$
D_{10} = D_{10} \times 80\%
$$

In [ ]:
model4 = None

In [ ]:
demand_q4 = {

1 : 280,
2 : 301,
3 : 277,
4 : 310,
5 : 285,
6 : 278,
7 : 291,
8 : 220 * 1.5 + 304 * 0.2 + 295 * 0.2,
9 : 304 * 0.8,
10 : 295 * 0.8,
11 : 302,
12 : 297
}


st_q4 = {

1 : 2600,
2 : 2600,
3 : 2600,
4 : 2600,
5 : 2600,
6 : 2600,
7 : 2600,
8 : 2340,
9 : 2600,
10 : 2600,
11 : 2600,
12 : 2600
}

In [ ]:
Tsales_q4 = lpSum(st_q4[i] * demand_q4[i] for i in months)

model4 = LpProblem("Aggregate-Planning-Problem", LpMaximize)
model4 += Tsales_q4 - Tlabor - Tinv - Tovertime - Tprod

In [ ]:
for i in months_inventory:

    if i == 0:
        model4 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model4 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand[i + 1]
    elif i == 12:
        model4 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model4 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand_q3[i+1], f"Inventory Balance {i}"


for i in months:

    model4 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"


for i in months:

    if i != 12:

        model4 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"


for i in months:

    model4 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:

    model4 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model4.solve()
LpStatus[model4.status]

'Optimal'

In [ ]:
for v in model4.variables():
    print(v.name, "=", v.varValue)

Hours_1 = 320.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 320.0
Hours_3 = 320.0
Hours_4 = 320.0
Hours_5 = 228.0
Hours_6 = 222.4
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 190.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 234.6
Inventory_3 = 317.6
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 25.6
Overtime_3 = 40.0
Overtime_4 = 40.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 320.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 345.6
Production_3 = 360.0
Production_4 = 360.0
Production_5 = 228.0
Production_6 = 222.4
Production_7 = 291.0
Production_8 = 220.0
Production_9 = 304.0


In [ ]:
print("Optimal objective function value = ", value(model4.objective))

Optimal objective function value =  1682432.0


# Question 5

$$
\text{Parameter Update}
$$

$$
s_t = $2600
$$

$$
D_4 = D_4 \times 50\%
$$


In [ ]:
demand_q5 = {

1 : 280,
2: 301,
3 : 277,
4: 310 * 0.5,
5 : 285,
6 : 278,
7 : 291,
8 : 220,
9 : 304,
10 : 295,
11 : 302,
12 : 297
}

In [ ]:
Tsales_q5 = lpSum(2600 * demand_q5[i] for i in months)

model5 = LpProblem("Aggregate-Planning-Problem", LpMaximize)
model5 += Tsales_q5 - Tlabor - Tinv - Tovertime - Tprod

In [ ]:
for i in months_inventory:

    if i == 0:
        model5 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model5 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand_q5[i + 1]
    elif i == 12:
        model5 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model5 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand_q5[i+1], f"Inventory Balance {i}"


for i in months:
    model5 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"

for i in months:
    if i != 12:
        model5 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

for i in months:
    model5 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model5 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model5.solve()
print("Optimal objective function value = ", value(model5.objective))
LpStatus[model5.status]

Optimal objective function value =  1827400.0


'Optimal'

In [ ]:
for v in model5.variables():
    print(v.name, "=", v.varValue)

Hours_1 = 230.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 301.0
Hours_3 = 277.0
Hours_4 = 155.0
Hours_5 = 285.0
Hours_6 = 278.0
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 100.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 100.0
Inventory_3 = 100.0
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 0.0
Overtime_3 = 0.0
Overtime_4 = 0.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 230.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 301.0
Production_3 = 277.0
Production_4 = 155.0
Production_5 = 285.0
Production_6 = 278.0
Production_7 = 291.0
Production_8 = 220.0
Production_9 = 304.0


# Question 6

$$
\text{Parameter Update}
$$

$$
s_t = $2600 \text{ except } s_4
$$

$$
s_4 = $2600 - $260 = $2340
$$

$$
D_4 = D_4 \times 150\% + D_5 \times 20\% + D_6 \times 20\%
$$

$$
D_5 = D_5 \times 80\%
$$

$$
D_6 = D_6 \times 80\%
$$

In [ ]:
demand_q6 = {

1 : 280,
2 : 301,
3 : 277,
4 : 310 * 1.5 + 285 * 0.2 + 278 * 0.2,
5 : 285 * 0.8,
6 : 278 * 0.8,
7 : 291,
8 : 220,
9 : 304,
10 : 295,
11 : 302,
12 : 297
}

st_q6 = {

1 : 2600,
2 : 2600,
3 : 2600,
4 : 2340,
5 : 2600,
6 : 2600,
7 : 2600,
8 : 2600,
9 : 2600,
10 : 2600,
11 : 2600,
12 : 2600
}

In [ ]:
Tsales_q6 = lpSum(st_q6[i] * demand_q6[i] for i in months)

model6 = LpProblem("Aggregate-Planning-Problem", LpMaximize)
model6 += Tsales_q6 - Tlabor - Tinv - Tovertime - Tprod

In [ ]:
for i in months_inventory:

    if i == 0:
        model6 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model6 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand_q6[i + 1]
    elif i == 12:
        model6 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model6 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand_q6[i+1], f"Inventory Balance {i}"

for i in months:
    model6 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"

for i in months:
    if i != 12:
        model6 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

for i in months:
    model6 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model6 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model6.solve()
LpStatus[model6.status]

'Optimal'

In [ ]:
print("Optimal objective function value = ", value(model6.objective))

Optimal objective function value =  1766204.0


In [ ]:
for v in model6.variables():
    print(v.name, "=", v.varValue)

Hours_1 = 320.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 320.0
Hours_3 = 320.0
Hours_4 = 320.0
Hours_5 = 228.0
Hours_6 = 222.4
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 190.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 234.6
Inventory_3 = 317.6
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 25.6
Overtime_3 = 40.0
Overtime_4 = 40.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 320.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 345.6
Production_3 = 360.0
Production_4 = 360.0
Production_5 = 228.0
Production_6 = 222.4
Production_7 = 291.0
Production_8 = 220.0
Production_9 = 304.0


# Question 7

$$
\text{Parameter Update}
$$

$$
s_t = $2600 \text{ except } s_4
$$

$$
s_4 = $2600 - $260 = $2340
$$

$$
D_4 = D_4 + D_5 \times 25\% + D_6 \times 25\%
$$

$$
D_5 = D_5 \times 75\%
$$

$$
D_6 = D_6 \times 75\%
$$

In [ ]:
demand_q7 = {

1 : 280,
2 : 301,
3 : 277,
4 : 310 + 285 * 0.25 + 278 * 0.25,
5 : 285 * 0.75,
6 : 278 * 0.75,
7 : 291,
8 : 220,
9 : 304,
10 : 295,
11 : 302,
12 : 297
}

st_q7 = {

1 : 2600,
2 : 2600,
3 : 2600,
4 : 2340,
5 : 2600,
6 : 2600,
7 : 2600,
8 : 2600,
9 : 2600,
10 : 2600,
11 : 2600,
12 : 2600
}

In [ ]:
Tsales_q7 = lpSum(st_q7[i] * demand_q7[i] for i in months)

model7 = LpProblem("Aggregate-Planning-Problem", LpMaximize)
model7 += Tsales_q7 - Tlabor - Tinv - Tovertime - Tprod

In [ ]:
for i in months_inventory:

    if i == 0:
        model7 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model7 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand_q7[i + 1]
    elif i == 12:
        model7 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model7 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand_q7[i+1], f"Inventory Balance {i}"

for i in months:
    model7 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"

for i in months:
    if i != 12:
        model7 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

for i in months:
    model7 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model7 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model7.solve()
LpStatus[model7.status]

'Optimal'

In [ ]:
print("Optimal objective function value = ", value(model7.objective))

Optimal objective function value =  1774480.0


In [ ]:
for v in model7.variables():
    print(v.name, "=", v.varValue)

Hours_1 = 298.75
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 320.0
Hours_3 = 320.0
Hours_4 = 320.0
Hours_5 = 213.75
Hours_6 = 208.5
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 168.75
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 187.75
Inventory_3 = 230.75
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 0.0
Overtime_3 = 0.0
Overtime_4 = 0.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 298.75
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 320.0
Production_3 = 320.0
Production_4 = 320.0
Production_5 = 213.75
Production_6 = 208.5
Production_7 = 291.0
Production_8 = 220.0
Production_9 = 304.0


# Question 8

$$
\text{Parameter Update}
$$

$$
s_t = $2600 \text{ except } s_8
$$

$$
s_8 = $2600 - $260 = $2340
$$

$$
D_8 = D_8 + D_9 \times 25\% + D_{10} \times 25\%
$$

$$
D_9 = D_9 \times 75\%
$$

$$
D_{10} = D_{10} \times 75\%
$$

In [ ]:
demand_q8 = {

1 : 280,
2 : 301,
3 : 277,
4 : 310,
5 : 285,
6 : 278,
7 : 291,
8 : 220 + 304 * 0.25 + 295 * 0.25,
9 : 304 * 0.75,
10 : 295 * 0.75,
11 : 302,
12 : 297
}

st_q8 = {

1 : 2600,
2 : 2600,
3 : 2600,
4 : 2600,
5 : 2600,
6 : 2600,
7 : 2600,
8 : 2340,
9 : 2600,
10 : 2600,
11 : 2600,
12 : 2600
}

In [ ]:
Tsales_q8 = lpSum(st_q8[i] * demand_q8[i] for i in months)

model8 = LpProblem("Aggregate-Planning-Problem", LpMaximize)
model8 += Tsales_q8 - Tlabor - Tinv - Tovertime - Tprod

for i in months_inventory:

    if i == 0:
        model8 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model8 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand_q8[i + 1]
    elif i == 12:
        model8 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model8 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand_q8[i+1], f"Inventory Balance {i}"

for i in months:
    model8 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"

for i in months:
    if i != 12:
        model8 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

for i in months:
    model8 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model8 += 40 >= overtime_hours[i], f"Overtime {i}"


model8.solve()
LpStatus[model8.status]

'Optimal'

In [ ]:
print("Optimal objective function value = ", value(model8.objective))

Optimal objective function value =  1817215.0


In [ ]:
for v in model8.variables():
    print(v.name, "=", v.varValue)

Hours_1 = 230.0
Hours_10 = 230.25
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 301.0
Hours_3 = 277.0
Hours_4 = 310.0
Hours_5 = 285.0
Hours_6 = 298.75
Hours_7 = 320.0
Hours_8 = 320.0
Hours_9 = 228.0
Inventory_0 = 150.0
Inventory_1 = 100.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 100.0
Inventory_3 = 100.0
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 120.75
Inventory_7 = 149.75
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 0.0
Overtime_3 = 0.0
Overtime_4 = 0.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 230.0
Production_10 = 230.25
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 301.0
Production_3 = 277.0
Production_4 = 310.0
Production_5 = 285.0
Production_6 = 298.75
Production_7 = 320.0
Production_8 = 320.0
Production_9 = 228.0


# Question 9

$$
\text{Parameter Update}
$$

$$
s_4 = $2600 - $260 = $2340
$$

$$
D_4 = D_4 \times 150\% + D_5 \times 20\% + D_6 \times 20\%
$$

$$
D_5 = D_5 \times 80\%
$$

$$
D_7 = D_7 \times 80\%
$$

$$
D_8 = D_8 \times 50\%
$$

In [ ]:
demand_q9 = {

1 : 280,
2: 301,
3 : 277,
4: 310 * 1.5 + 285 * 0.2 + 278 * 0.2,
5 : 285 * 0.8,
6 : 278 * 0.8,
7 : 291,
8 : 220 * 0.5,
9 : 304,
10 : 295,
11 : 302,
12 : 297
}

st_q9 = {

1 : 2600,
2 : 2600,
3 : 2600,
4 : 2340,
5 : 2600,
6 : 2600,
7 : 2600,
8 : 2600,
9 : 2600,
10 : 2600,
11 : 2600,
12 : 2600
}

In [ ]:
Tsales_q9 = lpSum(st_q9[i] * demand_q9[i] for i in months)

model9 = LpProblem("Aggregate-Planning-Problem", LpMaximize)
model9 += Tsales_q9 - Tlabor - Tinv - Tovertime - Tprod

In [ ]:
for i in months_inventory:

    if i == 0:
        model9 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model9 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand[i + 1]
    elif i == 12:
        model9 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model9 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand_q3[i+1], f"Inventory Balance {i}"


for i in months:
    model9 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"

for i in months:
    if i != 12:
        model9 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

for i in months:
    model9 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model9 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model9.solve()
LpStatus[model9.status]

'Optimal'

In [ ]:
print("Optimal objective function value = ", value(model9.objective))

Optimal objective function value =  1480204.0


In [ ]:
for v in model9.variables():
    print(v.name, "=", v.varValue)

Hours_1 = 320.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 320.0
Hours_3 = 320.0
Hours_4 = 320.0
Hours_5 = 228.0
Hours_6 = 222.4
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 190.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 234.6
Inventory_3 = 317.6
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 25.6
Overtime_3 = 40.0
Overtime_4 = 40.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 320.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 345.6
Production_3 = 360.0
Production_4 = 360.0
Production_5 = 228.0
Production_6 = 222.4
Production_7 = 291.0
Production_8 = 220.0
Production_9 = 304.0


# Question 10

$$
\text{Parameter Update}
$$

$$
s_8 = $2600 - $260 = $2340
$$

$$
D_8 = D_8 \times 150\% + D_9 \times 20\% + D_{10} \times 20\%
$$

$$
D_9 = D_9 \times 80\%
$$

$$
D_{10} = D_{10} \times 80\%
$$

$$
D_4 = D_8 \times 50\%
$$

In [ ]:
demand_q10 = {

1 : 280,
2 : 301,
3 : 277,
4: 310 * 0.5,
5 : 285 ,
6 : 278 ,
7 : 291,
8 : 220 * 1.5 + 304 * 0.2 + 295 * 0.2,
9 : 304,
10 : 295,
11 : 302,
12 : 297
}

st_q10 = {

1 : 2600,
2 : 2600,
3 : 2600,
4 : 2600,
5 : 2600,
6 : 2600,
7 : 2600,
8 : 2340,
9 : 2600,
10 : 2600,
11 : 2600,
12 : 2600
}

In [ ]:
Tsales_q10 = lpSum(st_q10[i] * demand_q10[i] for i in months)

model10 = LpProblem("Aggregate-Planning-Problem", LpMaximize)
model10 += Tsales_q10 - Tlabor - Tinv - Tovertime - Tprod

In [ ]:
for i in months_inventory:

    if i == 0:
        model10 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model10 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand[i + 1]
    elif i == 12:
        model10 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model10 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand_q3[i+1], f"Inventory Balance {i}"

for i in months:
    model10 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"


for i in months:
    if i != 12:
        model10 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

for i in months:
    model10 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model10 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model10.solve()
LpStatus[model10.status]

'Optimal'

In [ ]:
for v in model10.variables():
    print(v.name, "=", v.varValue)

Hours_1 = 320.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 320.0
Hours_3 = 320.0
Hours_4 = 320.0
Hours_5 = 228.0
Hours_6 = 222.4
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 190.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 234.6
Inventory_3 = 317.6
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 25.6
Overtime_3 = 40.0
Overtime_4 = 40.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 320.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 345.6
Production_3 = 360.0
Production_4 = 360.0
Production_5 = 228.0
Production_6 = 222.4
Production_7 = 291.0
Production_8 = 220.0
Production_9 = 304.0


In [ ]:
print("Optimal objective function value = ", value(model10.objective))

Optimal objective function value =  1590912.0


# Question 13 - 5

$$
\text{Parameter Update}
$$

$$
s_t = $2600
$$

$$
b_t = \text{ subcontracting cost in month t }
$$

$$
b_t = $2300
$$

$$
\text{New Decision Variable}
$$

$$
B_t : \text{subcontracted amount in month t}
$$



$$
New Model
$$

$$
\max \sum s_t ⋅ D_T - c_t \cdot I_t - r_t \cdot R_t - o_t \cdot O_t - p_t \cdot P_t - b_t \cdot B_t
$$

$$
s.t.
$$


$$
I_t = I_{t-1} + P_t - D_t
$$

$$
P_t = R_t + O_t
$$

$$
I_0 = 150
$$

$$
I_{12} = 150
$$

$$
I_t \geq 100 \quad \text{for} \quad t \in \{1, 2, 3, \dots, 11\}
$$

$$
R_t \leq 320
$$

$$
O_t \leq 40
$$




$$
R_t, I_t, P_t, O_t, B_t\geq 0
$$



In [ ]:
demand_13_5= {

1 : 280,
2: 301,
3 : 277,
4: 310 * 0.5,
5 : 285,
6 : 278,
7 : 291,
8 : 220,
9 : 304,
10 : 295,
11 : 302,
12 : 297
}

In [ ]:
# bt
buying_cost = 2300 # we called it buying cost to not confuse it with the sales price st

# Bt
buying_vars = LpVariable.dicts("Buying", months, 0)

Tbuying   = lpSum(buying_cost * buying_vars[i] for i in months) # monthly bought quantities from new supplier

Tsales_13_5 = lpSum(2600 * demand_13_5[i] for i in months)

model13_5 = LpProblem("Aggregate-Planning-Problem", LpMaximize)

model13_5 += Tsales_13_5 - Tlabor - Tinv - Tovertime - Tprod - Tbuying


In [ ]:
for i in months_inventory:

    if i == 0:
        model13_5 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model13_5 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] + buying_vars[i+1] - demand_13_5[i + 1]
    elif i == 12:
        model13_5 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model13_5 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] + buying_vars[i+1] - demand_13_5[i+1], f"Inventory Balance {i}"

for i in months:
    model13_5 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"

for i in months:
    if i != 12:
        model13_5 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

for i in months:
    model13_5 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model13_5 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model13_5.solve()
print("Optimal objective function value = ", value(model13_5.objective))
LpStatus[model13_5.status]

Optimal objective function value =  1827400.0


'Optimal'

In [ ]:
for v in model13_5.variables():
    print(v.name, "=", v.varValue)

Buying_1 = 0.0
Buying_10 = 0.0
Buying_11 = 0.0
Buying_12 = 0.0
Buying_2 = 0.0
Buying_3 = 0.0
Buying_4 = 0.0
Buying_5 = 0.0
Buying_6 = 0.0
Buying_7 = 0.0
Buying_8 = 0.0
Buying_9 = 0.0
Hours_1 = 230.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 301.0
Hours_3 = 277.0
Hours_4 = 155.0
Hours_5 = 285.0
Hours_6 = 278.0
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 100.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 100.0
Inventory_3 = 100.0
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 0.0
Overtime_3 = 0.0
Overtime_4 = 0.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 230.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 301.0
Production_3 = 277.0
Production_4 = 155.0
Pr

# Question 13 - 6

In [ ]:
demand_13_6 = {

1 : 280,
2 : 301,
3 : 277,
4 : 310 * 1.5 + 285 * 0.2 + 278 * 0.2,
5 : 285 * 0.8,
6 : 278 * 0.8,
7 : 291,
8 : 220,
9 : 304,
10 : 295,
11 : 302,
12 : 297
}

st_13_6 = {

1 : 2600,
2 : 2600,
3 : 2600,
4 : 2340,
5 : 2600,
6 : 2600,
7 : 2600,
8 : 2600,
9 : 2600,
10 : 2600,
11 : 2600,
12 : 2600
}

In [ ]:
# bt
buying_cost = 2300

# Bt
buying_vars = LpVariable.dicts("Buying", months, 0)

Tbuying   = lpSum(buying_cost * buying_vars[i] for i in months) # monthly bought quantities from new supplier

Tsales_13_6 = lpSum(st_13_6[i] * demand_13_6[i] for i in months)

model13_6 = LpProblem("Aggregate-Planning-Problem", LpMaximize)

model13_6 += Tsales_13_6 - Tlabor - Tinv - Tovertime - Tprod - Tbuying


In [ ]:
for i in months_inventory:

    if i == 0:
        model13_6 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model13_6 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] + buying_vars[i+1] - demand_13_6[i + 1]
    elif i == 12:
        model13_6 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model13_6 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] + buying_vars[i+1] - demand_13_6[i+1], f"Inventory Balance {i}"

for i in months:
    model13_6 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"


for i in months:
    if i != 12:
        model13_6 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"


for i in months:
    model13_6 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model13_6 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model13_6.solve()
print("Optimal objective function value = ", value(model13_6.objective))
LpStatus[model13_6.status]

Optimal objective function value =  1796444.0


'Optimal'

In [ ]:
for v in model13_6.variables():
    print(v.name, "=", v.varValue)

Buying_1 = 0.0
Buying_10 = 0.0
Buying_11 = 0.0
Buying_12 = 0.0
Buying_2 = 0.0
Buying_3 = 0.0
Buying_4 = 195.6
Buying_5 = 0.0
Buying_6 = 0.0
Buying_7 = 0.0
Buying_8 = 0.0
Buying_9 = 0.0
Hours_1 = 230.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 320.0
Hours_3 = 320.0
Hours_4 = 320.0
Hours_5 = 228.0
Hours_6 = 222.4
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 100.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 119.0
Inventory_3 = 162.0
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 0.0
Overtime_3 = 0.0
Overtime_4 = 0.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 230.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 320.0
Production_3 = 320.0
Production_4 = 320.0


# Question 13 - 7

In [ ]:
demand_13_7 = {

1 : 280,
2 : 301,
3 : 277,
4 : 310 + 285 * 0.25 + 278 * 0.25,
5 : 285 * 0.75,
6 : 278 * 0.75,
7 : 291,
8 : 220,
9 : 304,
10 : 295,
11 : 302,
12 : 297
}

st_13_7 = {

1 : 2600,
2 : 2600,
3 : 2600,
4 : 2340,
5 : 2600,
6 : 2600,
7 : 2600,
8 : 2600,
9 : 2600,
10 : 2600,
11 : 2600,
12 : 2600
}

In [ ]:
# bt
buying_cost = 2300

# Bt
buying_vars = LpVariable.dicts("Buying", months, 0)

Tbuying   = lpSum(buying_cost * buying_vars[i] for i in months) # monthly bought quantities from new supplier

Tsales_13_7 = lpSum(st_13_7[i] * demand_13_7[i] for i in months)

model13_7 = LpProblem("Aggregate-Planning-Problem", LpMaximize)

model13_7 += Tsales_13_7 - Tlabor - Tinv - Tovertime - Tprod - Tbuying



In [ ]:
for i in months_inventory:

    if i == 0:
        model13_7 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model13_7 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] + buying_vars[i+1] - demand_13_7[i + 1]
    elif i == 12:
        model13_7 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model13_7 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] + buying_vars[i+1] - demand_13_7[i+1], f"Inventory Balance {i}"

for i in months:
    model13_7 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"

for i in months:
    if i != 12:
        model13_7 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

for i in months:
    model13_7 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model13_7 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model13_7.solve()
print("Optimal objective function value = ", value(model13_7.objective))
LpStatus[model13_7.status]

Optimal objective function value =  1774480.0


'Optimal'

In [ ]:
for v in model13_7.variables():
    print(v.name, "=", v.varValue)

Buying_1 = 0.0
Buying_10 = 0.0
Buying_11 = 0.0
Buying_12 = 0.0
Buying_2 = 0.0
Buying_3 = 0.0
Buying_4 = 68.75
Buying_5 = 0.0
Buying_6 = 0.0
Buying_7 = 0.0
Buying_8 = 0.0
Buying_9 = 0.0
Hours_1 = 230.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 320.0
Hours_3 = 320.0
Hours_4 = 320.0
Hours_5 = 213.75
Hours_6 = 208.5
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 100.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 119.0
Inventory_3 = 162.0
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 0.0
Overtime_3 = 0.0
Overtime_4 = 0.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 230.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 320.0
Production_3 = 320.0
Production_4 = 320.0

# Question 13 - 8


In [ ]:
demand_13_8 = {

1 : 280,
2 : 301,
3 : 277,
4 : 310,
5 : 285,
6 : 278,
7 : 291,
8 : 220 + 304 * 0.25 + 295 * 0.25,
9 : 304 * 0.75,
10 : 295 * 0.75,
11 : 302,
12 : 297
}

st_13_8 = {

1 : 2600,
2 : 2600,
3 : 2600,
4 : 2600,
5 : 2600,
6 : 2600,
7 : 2600,
8 : 2340,
9 : 2600,
10 : 2600,
11 : 2600,
12 : 2600
}

In [ ]:
# bt
buying_cost = 2300

# Bt
buying_vars = LpVariable.dicts("Buying", months, 0)

Tbuying   = lpSum(buying_cost * buying_vars[i] for i in months) # monthly bought quantities from new supplier

Tsales_13_8 = lpSum(st_13_8[i] * demand_13_8[i] for i in months)

model13_8 = LpProblem("Aggregate-Planning-Problem", LpMaximize)

model13_8 += Tsales_13_8 - Tlabor - Tinv - Tovertime - Tprod - Tbuying


In [ ]:
for i in months_inventory:

    if i == 0:
        model13_8 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model13_8 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] + buying_vars[i+1] - demand_13_8[i + 1]
    elif i == 12:
        model13_8 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model13_8 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] + buying_vars[i+1] - demand_13_8[i+1], f"Inventory Balance {i}"

for i in months:
    model13_8 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"

for i in months:
    if i != 12:
        model13_8 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"


for i in months:
    model13_8 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model13_8 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model13_8.solve()
print("Optimal objective function value = ", value(model13_8.objective))
LpStatus[model13_8.status]

Optimal objective function value =  1817215.0


'Optimal'

In [ ]:
for v in model13_8.variables():
    print(v.name, "=", v.varValue)

Buying_1 = 0.0
Buying_10 = 0.0
Buying_11 = 0.0
Buying_12 = 0.0
Buying_2 = 0.0
Buying_3 = 0.0
Buying_4 = 0.0
Buying_5 = 0.0
Buying_6 = 0.0
Buying_7 = 0.0
Buying_8 = 0.0
Buying_9 = 0.0
Hours_1 = 230.0
Hours_10 = 230.25
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 301.0
Hours_3 = 277.0
Hours_4 = 310.0
Hours_5 = 285.0
Hours_6 = 298.75
Hours_7 = 320.0
Hours_8 = 320.0
Hours_9 = 228.0
Inventory_0 = 150.0
Inventory_1 = 100.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 100.0
Inventory_3 = 100.0
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 120.75
Inventory_7 = 149.75
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 0.0
Overtime_3 = 0.0
Overtime_4 = 0.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 230.0
Production_10 = 230.25
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 301.0
Production_3 = 277.0
Production_4 = 310

# Question 13-9

In [ ]:
# bt
buying_cost = 2300

buying_vars = LpVariable.dicts("Buying", months, 0)

Tsubcont = lpSum(buying_cost * buying_vars[i] for i in months)

In [ ]:
model13_9 = LpProblem("Aggregate-Planning-Problem", LpMaximize)
model13_9 += Tsales_q9 - Tlabor - Tinv - Tovertime - Tprod - Tsubcont

In [ ]:
for i in months_inventory:

    if i == 0:
        model13_9 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model13_9 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand[i + 1] + buying_vars[i + 1]
    elif i == 12:
        model13_9 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model13_9 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand_q3[i+1] + buying_vars[i + 1], f"Inventory Balance {i}"


for i in months:
    model13_9 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"

for i in months:
    if i != 12:

        model13_9 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

for i in months:
    model13_9 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model13_9 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model13_9.solve()
LpStatus[model13_9.status]

'Optimal'

In [ ]:
print("Optimal objective function value = ", value(model13_9.objective))

Optimal objective function value =  1510444.0


In [ ]:
for v in model13_9.variables():
    print(v.name, "=", v.varValue)

Buying_1 = 0.0
Buying_10 = 0.0
Buying_11 = 0.0
Buying_12 = 0.0
Buying_2 = 0.0
Buying_3 = 0.0
Buying_4 = 195.6
Buying_5 = 0.0
Buying_6 = 0.0
Buying_7 = 0.0
Buying_8 = 0.0
Buying_9 = 0.0
Hours_1 = 230.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 320.0
Hours_3 = 320.0
Hours_4 = 320.0
Hours_5 = 228.0
Hours_6 = 222.4
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 100.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 119.0
Inventory_3 = 162.0
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 0.0
Overtime_3 = 0.0
Overtime_4 = 0.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 230.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 320.0
Production_3 = 320.0
Production_4 = 320.0


# Question 13-10

In [ ]:
model13_10 = LpProblem("Aggregate-Planning-Problem", LpMaximize)
model13_10 += Tsales_q10 - Tlabor - Tinv - Tovertime - Tprod - Tsubcont

In [ ]:
for i in months_inventory:

    if i == 0:
        model13_10 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        model13_10 += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand[i + 1] + buying_vars[i + 1]
    elif i == 12:
        model13_10 += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        model13_10 += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand_q3[i+1] + buying_vars[i + 1] , f"Inventory Balance {i}"


for i in months:
    model13_10 += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"

for i in months:
    if i != 12:

        model13_10 += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

for i in months:
    model13_10 += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    model13_10 += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
model13_10.solve()
LpStatus[model13_10.status]

'Optimal'

In [ ]:
print("Optimal objective function value = ", value(model13_10.objective))

Optimal objective function value =  1621152.0


In [ ]:
for v in model13_10.variables():
    print(v.name, "=", v.varValue)

Buying_1 = 0.0
Buying_10 = 0.0
Buying_11 = 0.0
Buying_12 = 0.0
Buying_2 = 0.0
Buying_3 = 0.0
Buying_4 = 195.6
Buying_5 = 0.0
Buying_6 = 0.0
Buying_7 = 0.0
Buying_8 = 0.0
Buying_9 = 0.0
Hours_1 = 230.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 320.0
Hours_3 = 320.0
Hours_4 = 320.0
Hours_5 = 228.0
Hours_6 = 222.4
Hours_7 = 291.0
Hours_8 = 220.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 100.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 119.0
Inventory_3 = 162.0
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 0.0
Overtime_3 = 0.0
Overtime_4 = 0.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 230.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 320.0
Production_3 = 320.0
Production_4 = 320.0


# Additional Calculations for Question 11 & 12


$$
\text{Parameter Update}
$$

$$
s_t = $2600
$$

$$
D_4 = D_4 \times 50\%
$$


In [ ]:
modeladd = []

In [ ]:
demand_add = {

1 : 280,
2: 301,
3 : 277,
4: 310,
5 : 285,
6 : 278,
7 : 291,
8 : 220 * 0.5,
9 : 304,
10 : 295,
11 : 302,
12 : 297
}

In [ ]:
Tsales_add = lpSum(2600 * demand_add[i] for i in months)

modeladd = LpProblem("Aggregate-Planning-Problem", LpMaximize)
modeladd += Tsales_add - Tlabor - Tinv - Tovertime - Tprod

In [ ]:
for i in months_inventory:

    if i == 0:
        modeladd += inventory_vars[i] == 150,  f"Inventory Balance {i}"
        modeladd += inventory_vars[i+1] == inventory_vars[i] + production_vars[i + 1] - demand_add[i + 1]
    elif i == 12:
        modeladd += inventory_vars[i] == 150,  f"Inventory Balance {i}"
    else:
        modeladd += inventory_vars[i + 1] == inventory_vars[i] + production_vars[i+1] - demand_add[i+1], f"Inventory Balance {i}"


for i in months:
    modeladd += production_vars[i] == regular_hours[i] + overtime_hours[i], f"Production Workforce Relation {i}"

for i in months:
    if i != 12:
        modeladd += inventory_vars[i] >= 100, f"Intermediate Inventory Level {i}"

for i in months:
    modeladd += regular_hours[i] <= 320, f"Workforce Balance {i}"

for i in months:
    modeladd += 40 >= overtime_hours[i], f"Overtime {i}"

In [ ]:
modeladd.solve()
print("Optimal objective function value = ", value(modeladd.objective))
LpStatus[modeladd.status]

Optimal objective function value =  1869400.0


'Optimal'

In [ ]:
for v in modeladd.variables():
    print(v.name, "=", v.varValue)

Hours_1 = 230.0
Hours_10 = 304.0
Hours_11 = 320.0
Hours_12 = 320.0
Hours_2 = 301.0
Hours_3 = 277.0
Hours_4 = 310.0
Hours_5 = 285.0
Hours_6 = 278.0
Hours_7 = 291.0
Hours_8 = 110.0
Hours_9 = 304.0
Inventory_0 = 150.0
Inventory_1 = 100.0
Inventory_10 = 109.0
Inventory_11 = 127.0
Inventory_12 = 150.0
Inventory_2 = 100.0
Inventory_3 = 100.0
Inventory_4 = 100.0
Inventory_5 = 100.0
Inventory_6 = 100.0
Inventory_7 = 100.0
Inventory_8 = 100.0
Inventory_9 = 100.0
Overtime_1 = 0.0
Overtime_10 = 0.0
Overtime_11 = 0.0
Overtime_12 = 0.0
Overtime_2 = 0.0
Overtime_3 = 0.0
Overtime_4 = 0.0
Overtime_5 = 0.0
Overtime_6 = 0.0
Overtime_7 = 0.0
Overtime_8 = 0.0
Overtime_9 = 0.0
Production_1 = 230.0
Production_10 = 304.0
Production_11 = 320.0
Production_12 = 320.0
Production_2 = 301.0
Production_3 = 277.0
Production_4 = 310.0
Production_5 = 285.0
Production_6 = 278.0
Production_7 = 291.0
Production_8 = 110.0
Production_9 = 304.0
